In [ ]:
import sklearn
import random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tcn import TCN
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

np.random.seed(34)

train_data = pd.read_csv("dataset/train_FD001.txt", sep= "\s+", header = None)
test_data = pd.read_csv("dataset/test_FD001.txt", sep = "\s+", header = None)
true_rul = pd.read_csv("dataset/RUL_FD001.txt", sep = '\s+', header = None)

def process_targets(data_length, early_rul = None):
    if early_rul == None:
        return np.arange(data_length-1, -1, -1)
    else:
        early_rul_duration = data_length - early_rul
        if early_rul_duration <= 0:
            return np.arange(data_length-1, -1, -1)
        else:
            return np.append(early_rul*np.ones(shape = (early_rul_duration,)), np.arange(early_rul-1, -1, -1))

def process_input_data_with_targets(input_data, target_data = None, window_length = 1, shift = 1):
    num_batches = np.int_(np.floor((len(input_data) - window_length)/shift)) + 1
    num_features = input_data.shape[1]
    output_data = np.repeat(np.nan, repeats = num_batches * window_length * num_features).reshape(num_batches, window_length,
                                                                                                  num_features)
    if target_data is None:
        for batch in range(num_batches):
            output_data[batch,:,:] = input_data[(0+shift*batch):(0+shift*batch+window_length),:]
        return output_data
    else:
        output_targets = np.repeat(np.nan, repeats = num_batches)
        for batch in range(num_batches):
            output_data[batch,:,:] = input_data[(0+shift*batch):(0+shift*batch+window_length),:]
            output_targets[batch] = target_data[(shift*batch + (window_length-1))]
        return output_data, output_targets

def process_test_data(test_data_for_an_engine, window_length, shift, num_test_windows = 1):
    max_num_test_batches = np.int_(np.floor((len(test_data_for_an_engine) - window_length)/shift)) + 1
    if max_num_test_batches < num_test_windows:
        required_len = (max_num_test_batches -1)* shift + window_length
        batched_test_data_for_an_engine = process_input_data_with_targets(test_data_for_an_engine[-required_len:, :],
                                                                          target_data = None,
                                                                          window_length = window_length, shift = shift)
        return batched_test_data_for_an_engine, max_num_test_batches
    else:
        required_len = (num_test_windows - 1) * shift + window_length
        batched_test_data_for_an_engine = process_input_data_with_targets(test_data_for_an_engine[-required_len:, :],
                                                                          target_data = None,
                                                                          window_length = window_length, shift = shift)
        return batched_test_data_for_an_engine, num_test_windows


window_length = 10
shift = 1
early_rul = 125            
processed_train_data = []
processed_train_targets = []

num_test_windows = 5     
processed_test_data = []
num_test_windows_list = []

columns_to_be_dropped = [0,1,2,3,4,5,9,10,14,20,22,23]

train_data_first_column = train_data[0]
test_data_first_column = test_data[0]

# Scale data for all engines
scaler = MinMaxScaler(feature_range = (-1,1))
train_data = scaler.fit_transform(train_data.drop(columns = columns_to_be_dropped))
test_data = scaler.transform(test_data.drop(columns = columns_to_be_dropped))

train_data = pd.DataFrame(data = np.c_[train_data_first_column, train_data])
test_data = pd.DataFrame(data = np.c_[test_data_first_column, test_data])

num_train_machines = len(train_data[0].unique())
num_test_machines = len(test_data[0].unique())

# Process training and test data sepeartely as number of engines in training and test set may be different.
# As we are doing scaling for full dataset, we are not bothered by different number of engines in training and test set.

# Process trianing data
for i in np.arange(1, num_train_machines + 1):
    temp_train_data = train_data[train_data[0] == i].drop(columns = [0]).values
    
    # Verify if data of given window length can be extracted from training data
    if (len(temp_train_data) < window_length):
        print("Train engine {} doesn't have enough data for window_length of {}".format(i, window_length))
        raise AssertionError("Window length is larger than number of data points for some engines. "
                             "Try decreasing window length.")
        
    temp_train_targets = process_targets(data_length = temp_train_data.shape[0], early_rul = early_rul)
    data_for_a_machine, targets_for_a_machine = process_input_data_with_targets(temp_train_data, temp_train_targets, 
                                                                                window_length = window_length, shift = shift)
    
    processed_train_data.append(data_for_a_machine)
    processed_train_targets.append(targets_for_a_machine)

processed_train_data = np.concatenate(processed_train_data)
processed_train_targets = np.concatenate(processed_train_targets)

# Process test data
for i in np.arange(1, num_test_machines + 1):
    temp_test_data = test_data[test_data[0] == i].drop(columns = [0]).values
    
    # Verify if data of given window length can be extracted from test data
    if (len(temp_test_data) < window_length):
        print("Test engine {} doesn't have enough data for window_length of {}".format(i, window_length))
        raise AssertionError("Window length is larger than number of data points for some engines. "
                             "Try decreasing window length.")
    
    # Prepare test data
    test_data_for_an_engine, num_windows = process_test_data(temp_test_data, window_length = window_length, shift = shift,
                                                             num_test_windows = num_test_windows)
    
    processed_test_data.append(test_data_for_an_engine)
    num_test_windows_list.append(num_windows)

processed_test_data = np.concatenate(processed_test_data)
true_rul = true_rul[0].values

# Shuffle training data
index = np.random.permutation(len(processed_train_targets))
processed_train_data, processed_train_targets = processed_train_data[index], processed_train_targets[index]

# print("Processed trianing data shape: ", processed_train_data.shape)
# print("Processed training ruls shape: ", processed_train_targets.shape)
# print("Processed test data shape: ", processed_test_data.shape)
# print("True RUL shape: ", true_rul.shape)

processed_train_data, processed_val_data, processed_train_targets, processed_val_targets = train_test_split(processed_train_data,
                                                                                                            processed_train_targets,
                                                                                                            test_size = 0.2,
                                                                                                            random_state = 83)
# print("Processed train data shape: ", processed_train_data.shape)
# print("Processed validation data shape: ", processed_val_data.shape)
# print("Processed train targets shape: ", processed_train_targets.shape)
# print("Processed validation targets shape: ", processed_val_targets.shape)

In [ ]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

# 创建TCN模型
def create_tcn_model():
    model = tf.keras.Sequential([
    TCN(input_shape=[window_length,processed_train_data.shape[2]],
            nb_filters=32,
            kernel_size=2,
            nb_stacks=1,
            dilations=(1, 2, 4, 8),
            padding='causal',
            use_skip_connections=True,
            dropout_rate=0.05,
            return_sequences=False,
            activation='relu',
            kernel_initializer='he_normal',
            use_batch_norm=False,
            use_layer_norm=True,
            use_weight_norm=False),
            tf.keras.layers.Dense(1)
            ])
    model.compile(loss = "mse", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),metrics=[rmse])
    return model

# 交叉函数
def crossover(weights1, weights2):
    crossover_point = random.randint(0, len(weights1))
    weights1[:crossover_point], weights2[:crossover_point] = weights2[:crossover_point], weights1[:crossover_point]
    return weights1

# 变异函数
def mutate(weights):
    t = random.randint(0, len(weights)-1)
    weights[t] *= 0.9 
    return weights

#计算RMSE
def get_RMSE(model):
    rul_pred = model.predict(processed_test_data).reshape(-1)
    preds_for_each_engine = np.split(rul_pred, np.cumsum(num_test_windows_list)[:-1])
    mean_pred_for_each_engine = [np.average(ruls_for_each_engine, weights = np.repeat(1/num_windows, num_windows)) 
                             for ruls_for_each_engine, num_windows in zip(preds_for_each_engine, num_test_windows_list)]
    RMSE = np.sqrt(mean_squared_error(true_rul, mean_pred_for_each_engine))
    return RMSE

def get_RMSE_train(model):
    rul_pred = model.predict(processed_train_data).reshape(-1)
    preds_for_each_engine = np.split(rul_pred, np.cumsum(num_test_windows_list)[:-1])
    mean_pred_for_each_engine = [np.average(ruls_for_each_engine, weights = np.repeat(1/num_windows, num_windows)) 
                             for ruls_for_each_engine, num_windows in zip(preds_for_each_engine, num_test_windows_list)]
    RMSE = np.sqrt(mean_squared_error(true_rul, mean_pred_for_each_engine))
    return RMSE

def compute_s_score(rul_true, rul_pred):
    diff = rul_pred - rul_true
    return np.sum(np.where(diff < 0, np.exp(-diff/13)-1, np.exp(diff/10)-1))

In [ ]:
# 定义种群
pop_size = 25
pop = []

# 定义遗传算法的参数
parent_selection_pressure = 0.5
mutation_rate = 0.1
loss_tar = 13.5

predict_train = []
predict_val = []

# 初始化权重
ori_weights = []
for i in range(pop_size):
    tf.keras.backend.clear_session()
    model = create_tcn_model()
    ori_weights.append(model.get_weights())

# 生成第一代
print("第1代")
fitness = []
for i in range(pop_size):
    model.set_weights(ori_weights[i])
    model.compile(optimizer='adam', loss='mse',metrics=[rmse])
    print(f"Individual_{i+1}:")
    model.fit(processed_train_data, processed_train_targets, epochs = 4,
                    validation_data = (processed_val_data, processed_val_targets),
                    batch_size = 16, verbose = 1)
    loss = get_RMSE(model)
    pop.append(model.get_weights())
    fitness.append(1/loss)

model.set_weights(pop[np.argmax(fitness)])

generation = 1
    
num_generations = 10  
for generation in range(2, num_generations + 1):
    #选择父母
    parents = []
    for i in range(pop_size):
        #轮盘赌徒法
        idx1 = np.random.choice(np.arange(pop_size), p=fitness/np.sum(fitness))
        idx2 = np.random.choice(np.arange(pop_size), p=fitness/np.sum(fitness))
        if fitness[idx1] > fitness[idx2]:
            parents.append(pop[idx1])
        else:
            parents.append(pop[idx2])
    # 产生新的后代
    offspring = []
    for i in range(pop_size):
        # 交叉
        if np.random.random() < parent_selection_pressure:
            weight1 = parents[i]
            weight2 = parents[(i+1) % pop_size]
            offspring.append(crossover(weight1,weight2))
        # 变异
        elif np.random.random() < mutation_rate:
            offspring.append(mutate(parents[i]))
        # 直接复制
        else:
            offspring.append(parents[i])

    # 替换种群  
    pop = offspring

    # 计算RMSE
    print(f"第{generation}代")
    fitness = []

    for i in range(pop_size):
        model.set_weights(pop[i])
        model.compile(optimizer='adam', loss='mse',metrics=[rmse])
        print(f"Individual_{i+1}:")
        model.fit(processed_train_data, processed_train_targets, epochs = 4,
                        validation_data = (processed_val_data, processed_val_targets),
                        batch_size = 16, verbose = 1)
        loss = get_RMSE(model)
        pop[i] = model.get_weights()
        fitness.append(1/loss)



    # 选择最优解
    model.set_weights(pop[np.argmax(fitness)])
    loss = get_RMSE(model)
    print(loss)


print(f"best loss:{loss}")

In [ ]:
rul_pred = model.predict(processed_test_data).reshape(-1)
preds_for_each_engine = np.split(rul_pred, np.cumsum(num_test_windows_list)[:-1])
mean_pred_for_each_engine = [np.average(ruls_for_each_engine, weights = np.repeat(1/num_windows, num_windows)) 
                             for ruls_for_each_engine, num_windows in zip(preds_for_each_engine, num_test_windows_list)]
MAE = mean_absolute_error(true_rul, mean_pred_for_each_engine)
MSE =  mean_squared_error(true_rul, mean_pred_for_each_engine)
RMSE = np.sqrt(mean_squared_error(true_rul, mean_pred_for_each_engine))
print("MAE: ", MAE)
print("MSE: ", MSE)
print("RMSE: ", RMSE)

In [ ]:
indices_of_last_examples = np.cumsum(num_test_windows_list) - 1
preds_for_last_example = np.concatenate(preds_for_each_engine)[indices_of_last_examples]

RMSE_new = np.sqrt(mean_squared_error(true_rul, preds_for_last_example))
print("RMSE (Taking only last examples): ", RMSE_new)

s_score = compute_s_score(true_rul, preds_for_last_example)
print("S-score: ", s_score)

In [ ]:
#########################